<a href="https://colab.research.google.com/github/fatemafaria142/Recipe-Generation-with-GPT-2-Evaluating-Generated-Recipes-Against-Reference-Recipes/blob/main/Recipe_Generation_using_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Read dataset from google drive**
# **Dataset Link: https://huggingface.co/datasets/recipe_nlg**
# **Paper Link: https://aclanthology.org/2020.inlg-1.4/**

In [3]:
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/Recipe Generation/full_dataset.csv')
dataset.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


# **Length of the main dataset**

In [4]:
len(dataset)

2231142

# **Take subset of the main dataset**

In [5]:
# Extracting the first 15,000 examples for train and test
data = dataset.head(15000)

# Saving the subset as a CSV file
data.to_csv('/content/drive/MyDrive/Recipe Generation/recipes.csv', index=False)


# **Taking 90% for train dataset and 10% for test dataset**

In [6]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the dataset from the CSV file
data = pd.read_csv('/content/drive/MyDrive/Recipe Generation/recipes.csv')

def build_text_files(data_csv, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for index, row in data_csv.iterrows():
        summary = str(row['directions']).strip()
        summary = re.sub(r"[\[\]\"]", "", summary)  # Remove square brackets and quotation marks
        summary = re.sub(r"\s+", " ", summary)  # Replace multiple spaces with a single space
        data += summary + "  "
    f.write(data)

train, test = train_test_split(data, test_size=0.10)

build_text_files(train, '/content/drive/MyDrive/Recipe Generation/train_dataset.txt')
build_text_files(test, '/content/drive/MyDrive/Recipe Generation/test_dataset.txt')

print("Train dataset length:", len(train))
print("Test dataset length:", len(test))


Train dataset length: 13500
Test dataset length: 1500


# **Text Preprocessing**

In [7]:
def read_text_file(file_path, num_examples=5):
    with open(file_path, 'r') as file:
        content = file.read()
        examples = content.split('  ')  # Assuming '  ' is the separator between examples
        for i, example in enumerate(examples[:num_examples]):
            print(f"Example {i+1}: {example}")

train_file_path = '/content/drive/MyDrive/Recipe Generation/train_dataset.txt'
test_file_path = '/content/drive/MyDrive/Recipe Generation/test_dataset.txt'

print("Examples from the Train Dataset:")
read_text_file(train_file_path)

print("\nExamples from the Test Dataset:")
read_text_file(test_file_path)


Examples from the Train Dataset:
Example 1: Preheat oven to 375\u00b0., In a large bowl, combine flour, granulated sugar, baking powder, 1/2 teaspoon cinnamon and salt. In another bowl, beat egg with milk, then stir in apple and melted butter., Add all at once to flour mixture and stir until just moistened., Spoon batter into a greased muffin tin(s), filling each cup about 2/3 full.
Example 2: Mix all ingredients except soda., Boil until a peanut tastes done when removed and cooled in a cup of water., When the peanuts are done, remove the pan from heat and add a pinch of soda., (Candy will foam up.), Stir well and spread out on a greased cookie sheet. When cool, break up into small pieces.
Example 3: Prepare cake as directed and bake in a greased and floured 9 x 13-inch pan., Meanwhile, simmer pineapple and sugar., Remove baked cake from oven and, with a fork, poke holes all over cake., Cover with pineapple mixture and chill., Prepare pudding as directed and stir in coconut., Spread ov

# **GPT2 and its tokenizer**
# **https://huggingface.co/gpt2**

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

In [19]:
train_path = '/content/drive/MyDrive/Recipe Generation/train_dataset.txt'
test_path = '/content/drive/MyDrive/Recipe Generation/test_dataset.txt'

# **Prepare the dataset and build a TextDataset**
* The next step is to extract the instructions from all recipes and build a TextDataset. The TextDataset is a custom implementation of the Pytroch Dataset class implemented by the transformers library.

In [25]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

In [26]:
!pip install transformers[torch]

In [27]:
!pip install accelerate -U

In [28]:
!pip install --upgrade transformers

# **1st time train - Initialize Trainer with TrainingArguments**

In [48]:
'''
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-englishchef", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
'''

'\nfrom transformers import Trainer, TrainingArguments\n\ntraining_args = TrainingArguments(\n    output_dir="./gpt2-englishchef", #The output directory\n    overwrite_output_dir=True, #overwrite the content of the output directory\n    num_train_epochs=3, # number of training epochs\n    per_device_train_batch_size=32, # batch size for training\n    per_device_eval_batch_size=64,  # batch size for evaluation\n    eval_steps = 400, # Number of update steps between two evaluations.\n    save_steps=800, # after # steps model is saved\n    warmup_steps=500,# number of warmup steps for learning rate scheduler\n    prediction_loss_only=True,\n    )\n\n\ntrainer = Trainer(\n    model=model,\n    args=training_args,\n    data_collator=data_collator,\n    train_dataset=train_dataset,\n    eval_dataset=test_dataset,\n)\n'

In [49]:
'''
trainer.train()
'''

'\ntrainer.train()\n'

In [50]:
'''
trainer.save_model()
'''

'\ntrainer.save_model()\n'

# **Loading the saved model and train it again**

* The Trainer class provides an API for feature-complete training. It is used in most of the example scripts from Huggingface. Before we can instantiate our Trainer we need to download our GPT-2 model and create a TrainingArguments to access all the points of customization during training. In the TrainingArguments, we can define the Hyperparameters we are going to use in the training process like our learning_rate, num_train_epochs, or per_device_train_batch_size.


In [39]:
from transformers import Trainer, TrainingArguments
# Load the previously trained model
model_path = "./gpt2-englishchef"  # Replace this with the path to your saved model
model = AutoModelForCausalLM.from_pretrained(model_path)

# Define your training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-englishchef",  # The output directory
    overwrite_output_dir=True,  # Overwrite the content of the output directory
    num_train_epochs=5,  # Number of training epochs
    per_device_train_batch_size=32,  # Batch size for training
    per_device_eval_batch_size=64,  # Batch size for evaluation
    eval_steps=400,  # Number of update steps between two evaluations
    save_steps=800,  # Save model checkpoint after # steps
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
)

# Initialize Trainer with your loaded model and training arguments
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


# **Train the model**

In [40]:
# Start training to continue from where it left off
trainer.train()

Step,Training Loss
500,2.348800


TrainOutput(global_step=990, training_loss=2.3012694079466542, metrics={'train_runtime': 1111.7223, 'train_samples_per_second': 28.375, 'train_steps_per_second': 0.891, 'total_flos': 2060614287360000.0, 'train_loss': 2.3012694079466542, 'epoch': 5.0})

# **Save the model**

In [41]:
# Save the model after training
trainer.save_model("./gpt2-englishchef")

# **Test the model**
* To test the model we are going to use another highlight of the transformers library called pipeline. Pipelines are objects that offer a simple API dedicated to several tasks, among others also text-generation

In [42]:
from transformers import pipeline

In [43]:
chef = pipeline('text-generation',model='./gpt2-englishchef', tokenizer='gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [44]:
chef('Beat together eggs, sugar and flour.')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Beat together eggs, sugar and flour. Gradually add milk to first mixture of flour, stirring constantly., Add vanilla. Beat for 2 minutes., Add eggs, blend for 2 minutes., Fold in fruit and pour into fruit mixture., Pour filling into pie'}]

In [45]:
chef(' Stir Velveeta and salsa in saucepan over low heat until Velveeta is melted.')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': ' Stir Velveeta and salsa in saucepan over low heat until Velveeta is melted. Add chopped olives to saucepan while they cool., Add tomatoes and saute until tomatoes are tender., Add tomatoes and cook until tomatoes are heated up ('}]

In [46]:
chef('In one bowl, combine all ingredients except frosting.')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In one bowl, combine all ingredients except frosting. In another bowl put 2 tablespoons of margarine and enough water to cover and refrigerate for at least 2 hours before serving. This prevents the spread from turning brown. Refrigerate until ready to'}]

In [47]:
chef(' Mix all ingredients except soda.')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Mix all ingredients except soda. Pour in ungreased pan and bake at 350 degrees for 20 minutes.  Combine all dry ingredients alternately with 1/2 cup sugar in large mixing bowl., Add yeast slowly and slowly., Mix vigorously with hands.,'

# **Evaluate perplexity on the test dataset**

In [51]:
# Evaluate perplexity on the test dataset
eval_result = trainer.evaluate(eval_dataset=test_dataset)
print("Perplexity:", eval_result['eval_loss'])

Perplexity: 2.2754464149475098


# **Generating Recipes without any prompt**

In [59]:
from transformers import pipeline

# Initialize the text generation pipeline
chef = pipeline('text-generation', model='./gpt2-englishchef', tokenizer='gpt2')

# Function to generate recipes and save them
def generate_recipes(chef_pipeline, num_recipes: int):
    generated_recipes = []
    for _ in range(num_recipes):
        generated_recipe = chef_pipeline("Recipe: ")  # You can modify the initial prompt if needed
        generated_recipes.append(generated_recipe[0]['generated_text'])
    return generated_recipes

# Generate recipes
num_recipes_to_generate = 5  # Define the number of recipes to generate
generated_recipes = generate_recipes(chef, num_recipes_to_generate)

# Display the generated recipes
for i, recipe in enumerate(generated_recipes, start=1):
    print(f"Generated Recipe {i}:\n{recipe}\n")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Recipe 1:
Recipe:  In skillet, brown onion and red pepper in chicken broth., Add soup and soy sauce., Simmer through; add remaining ingredients. Cover; simmer, uncovered, for 45 minutes. Sprinkle with remaining chives and bake in greased

Generated Recipe 2:
Recipe:  Brown fat., Remove from skillet., Stir in milk, green chilies and salt. Add egg white and cook until thick, about 35 to 35 minutes. Stir in rice. Cook on greased cookie sheet. Transfer to serving bowl.,

Generated Recipe 3:
Recipe:  Sift dry ingredients together., Place 1 cup pineapple and 2-cup nuts in bowl. Add 3-5 of sugar; mix, then spoon and coat with margarine., Add 2-ounces of hot water (1/2 cup

Generated Recipe 4:
Recipe:  Mix and let cook. Makes 24 servings.  Boil broccoli; drain well. Brown ground beef and onion in skillet., Then add kidney beans., Then pickle shrimp and then sauerkraut sauce., Simmer for 35

Generated Recipe 5:
Recipe:  In a 1 1/2-quart baking dish, mix 1 stick margarine, oil, garlic 

# **Examples from Test Dataset**

In [66]:
def read_text_file(file_path, num_examples=5):
    with open(file_path, 'r') as file:
        content = file.read()
        examples = content.split('  ')  # Assuming '  ' is the separator between examples
        for i, example in enumerate(examples[:num_examples]):
            print(f"Example {i+1}: {example}")


test_file_path = '/content/drive/MyDrive/Recipe Generation/test_dataset.txt'

print("\nExamples from the Test Dataset:")
read_text_file(test_file_path)


Examples from the Test Dataset:
Example 1: Make and, chill, crust., Combine, 1, 1/2, cups packaged graham cracker crumbs and 1/4 cup sugar in a small bowl; blend in 1/2 cup (1 stick) melted butter or margarine., Press firmly over bottom and, sides of a heavy buttered 9-inch spring-form pan., Chill.
Example 2: Stir Velveeta and salsa in saucepan over low heat until Velveeta is melted., Stir in cilantro., Serve hot with tortilla chips or vegetable dippers., Makes 3 cups.
Example 3: Beat together eggs, sugar and flour., Combine with corn, butter, salt and milk., Pour into a greased 1-quart casserole. Bake at 300\u00b0 for 50 minutes., Yields 4 servings.
Example 4: Use 1 1/2-quart buttered casserole., Preheat oven to 350\u00b0.
Example 5: In one bowl, combine all ingredients except frosting., Beat with mixer at high speed for 2 minutes., Grease and flour medium size (9 cup) Bundt pan and pour in batter., (This will give a high cake; larger pans will give a flatter cake.), Bake at 350\u00b

# **Generating Recipes with prompt from test dataset**

In [74]:
from transformers import pipeline
import random

test_dataset_file = '/content/drive/MyDrive/Recipe Generation/test_dataset.txt'

# Initialize the text generation pipeline
chef = pipeline('text-generation', model='./gpt2-englishchef', tokenizer='gpt2')

# Read your test dataset
with open(test_dataset_file, 'r') as file:
    test_content = file.read()

# Split the content into individual recipes based on the delimiter ('  ')
test_recipes = test_content.split('  ')

# Function to generate recipes based on the test dataset
def generate_recipes_from_test_dataset(chef_pipeline, test_dataset, num_recipes: int):
    generated_recipes = []
    for i in range(num_recipes):
        # Randomly select a recipe from the test dataset
        random_recipe = random.choice(test_dataset)
        # Extract the first 6/7 words from the recipe as the prompt
        words = random_recipe.split()[:6]  # Change the number here to select the desired number of words
        prompt = ' '.join(words)
        generated_recipe = chef_pipeline(prompt)
        generated_recipes.append(generated_recipe[0]['generated_text'])
    return generated_recipes

# Generate recipes based on the test dataset
num_recipes_to_generate = 5  # Define the number of recipes to generate
generated_recipes_from_test = generate_recipes_from_test_dataset(chef, test_recipes, num_recipes_to_generate)

# Display the generated recipes
for i, recipe in enumerate(generated_recipes_from_test, start=1):
    print(f"Generated Recipe {i}:\n{recipe}\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Recipe 1:
About 40 minutes before serving or serving, spread a layer of cheese over top., Slice tomatoes in half., Place a slice of sausage atop top., Spread soup mix on sausage., Place a slice of cheese atop top of tomato mixture., Repeat. 

Generated Recipe 2:
Mix until crumbly., Press into 9 x 13 inch pan.  In a 13 x 9 x 2-inch pan, melt margarine., Melt mayonnaise, butter and then cracker crumbs., Mix sugar and Crisco together.,

Generated Recipe 3:
Preheat oven to 350\u00b0., Sift first 7 ingredients together and add to bowl; beat until well mixed. Blend in sugar and flour and mix well. Add flour mixture first and mix well. Spoon 1/2 cup melted

Generated Recipe 4:
You will also need artificial holly rhubarb., Keep in water.  Mix sugar, cornstarch, and water to make syrup., Gradually add 1 cup of the boiling hot water., Add fruit, cherries, cherries, oranges

Generated Recipe 5:
Boil ribs until tender., Mix barbecue sauce and 1/4 cup cheese with a spoon, then combine rem

# **List of 1st 20 examples from Test Dataset**

In [76]:
def read_text_file(file_path, num_examples=5):
    with open(file_path, 'r') as file:
        content = file.read()
        examples = content.split('  ')  # Assuming '  ' is the separator between examples
        return examples[:num_examples]

test_file_path = '/content/drive/MyDrive/Recipe Generation/test_dataset.txt'

print("\nExamples from the Test Dataset:")
first_20_examples = read_text_file(test_file_path, num_examples=20)

# Display the first 20 examples
for i, example in enumerate(first_20_examples, start=1):
    print(f"Example {i}: {example}")

# Store the first 20 examples in a list
list_of_20_examples = first_20_examples


Examples from the Test Dataset:
Example 1: Make and, chill, crust., Combine, 1, 1/2, cups packaged graham cracker crumbs and 1/4 cup sugar in a small bowl; blend in 1/2 cup (1 stick) melted butter or margarine., Press firmly over bottom and, sides of a heavy buttered 9-inch spring-form pan., Chill.
Example 2: Stir Velveeta and salsa in saucepan over low heat until Velveeta is melted., Stir in cilantro., Serve hot with tortilla chips or vegetable dippers., Makes 3 cups.
Example 3: Beat together eggs, sugar and flour., Combine with corn, butter, salt and milk., Pour into a greased 1-quart casserole. Bake at 300\u00b0 for 50 minutes., Yields 4 servings.
Example 4: Use 1 1/2-quart buttered casserole., Preheat oven to 350\u00b0.
Example 5: In one bowl, combine all ingredients except frosting., Beat with mixer at high speed for 2 minutes., Grease and flour medium size (9 cup) Bundt pan and pour in batter., (This will give a high cake; larger pans will give a flatter cake.), Bake at 350\u00b

In [80]:
print(list_of_20_examples)

['Make and, chill, crust., Combine, 1, 1/2, cups packaged graham cracker crumbs and 1/4 cup sugar in a small bowl; blend in 1/2 cup (1 stick) melted butter or margarine., Press firmly over bottom and, sides of a heavy buttered 9-inch spring-form pan., Chill.', 'Stir Velveeta and salsa in saucepan over low heat until Velveeta is melted., Stir in cilantro., Serve hot with tortilla chips or vegetable dippers., Makes 3 cups.', 'Beat together eggs, sugar and flour., Combine with corn, butter, salt and milk., Pour into a greased 1-quart casserole. Bake at 300\\u00b0 for 50 minutes., Yields 4 servings.', 'Use 1 1/2-quart buttered casserole., Preheat oven to 350\\u00b0.', 'In one bowl, combine all ingredients except frosting., Beat with mixer at high speed for 2 minutes., Grease and flour medium size (9 cup) Bundt pan and pour in batter., (This will give a high cake; larger pans will give a flatter cake.), Bake at 350\\u00b0., Test at 45 minutes and then every 5 minutes until done., Cool in pa

In [79]:
print(len(list_of_20_examples))

20


In [77]:
from transformers import pipeline

test_dataset_file = '/content/drive/MyDrive/Recipe Generation/test_dataset.txt'

# Initialize the text generation pipeline
chef = pipeline('text-generation', model='./gpt2-englishchef', tokenizer='gpt2')

# Read your test dataset
with open(test_dataset_file, 'r') as file:
    test_content = file.read()

# Split the content into individual recipes based on the delimiter ('  ')
test_recipes = test_content.split('  ')

# Function to generate recipes based on the test dataset
def generate_recipes_from_test_dataset(chef_pipeline, test_dataset, num_recipes: int):
    generated_recipes = []
    for i in range(num_recipes):
        # Extract the first 6 words from the recipe as the prompt
        words = test_dataset[i].split()[:6] if i < len(test_dataset) else test_dataset[-1].split()[:6]
        prompt = ' '.join(words)
        generated_recipe = chef_pipeline(prompt, max_length=128)  # Adjust max_length as needed
        generated_recipes.append(generated_recipe[0]['generated_text'])
    return generated_recipes

# Generate recipes based on the test dataset
num_recipes_to_generate = 20  # Define the number of recipes to generate
generated_recipes_from_test = generate_recipes_from_test_dataset(chef, test_recipes, num_recipes_to_generate)

# Display the generated recipes
for i, recipe in enumerate(generated_recipes_from_test, start=1):
    print(f"Generated Recipe {i}:\n{recipe}\n")

# Save the generated recipes in a list
list_of_generated_recipes = generated_recipes_from_test


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 fo

Generated Recipe 1:
Make and, chill, crust., Combine, 1, teaspoon vanilla and 1 1/2 teaspoons oil in a large bowl; beat, together 1 beaten egg and 1 teaspoon cinnamon mixture until, blended, moistened., Stir in remaining vanilla and cinnamon mixture; fold in crumbled apples; and press into a 9 x 13-inch pan and bake at 350\u00b0 for 40 minutes.  Preheat oven to 400\u00b0., Prepare pastry shell by rolling into 1/8-inch balls., Put a layer of pastry in the first set of rings, then in the other ring, covering the rings; spread

Generated Recipe 2:
Stir Velveeta and salsa in saucepan until light and fluffy., Combine chips, margarine, soup, cheese, salt and pepper., Pour over veggies and mix until cheese and soup mixture is coated., Cool in refrigerator for about 5 minutes, watching closely for chip flavors to harden., Serve with lettuce and tomato if desired.  Cook broccoli, drain., Mix broccoli, broccoli and onion with eggs, curry powder and Parmesan cheese., Blend in crumbled bacon., Pou

In [78]:
print(list_of_generated_recipes)

['Make and, chill, crust., Combine, 1, teaspoon vanilla and 1 1/2 teaspoons oil in a large bowl; beat, together 1 beaten egg and 1 teaspoon cinnamon mixture until, blended, moistened., Stir in remaining vanilla and cinnamon mixture; fold in crumbled apples; and press into a 9 x 13-inch pan and bake at 350\\u00b0 for 40 minutes.  Preheat oven to 400\\u00b0., Prepare pastry shell by rolling into 1/8-inch balls., Put a layer of pastry in the first set of rings, then in the other ring, covering the rings; spread', 'Stir Velveeta and salsa in saucepan until light and fluffy., Combine chips, margarine, soup, cheese, salt and pepper., Pour over veggies and mix until cheese and soup mixture is coated., Cool in refrigerator for about 5 minutes, watching closely for chip flavors to harden., Serve with lettuce and tomato if desired.  Cook broccoli, drain., Mix broccoli, broccoli and onion with eggs, curry powder and Parmesan cheese., Blend in crumbled bacon., Pour in casserole dish., Bake at 350\

In [82]:
print(len(list_of_generated_recipes))

20


# **Evaluating Generated Recipes Against Reference Recipes**

In [84]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=92b35cf257364adf1909f38609a307fa21c3e207b33a1d779fc5e2e741565198
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [92]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00


In [96]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.3 MB/s eta 0:00:00


In [97]:
from evaluate import load
# Load the evaluation metric for Character Error Rate (CER) and Word Error Rate (WER) and Exact Match(em)
cer_metric = load("cer")
wer_metric = load("wer")
meteor = load('meteor')
exact_match_metric = load("exact_match")

# Load BLEU and ROUGE metrics
bleu_metric = load("bleu")
rouge_metric = load('rouge')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [98]:
# Calculate Character Error Rate (CER) and Word Error Rate (WER)
results_CER = cer_metric.compute(predictions=list_of_generated_recipes, references=list_of_20_examples)
results_WER = wer_metric.compute(predictions=list_of_generated_recipes, references=list_of_20_examples)

# Calculate Exact Match (EM) and METEOR(M)
results_em = exact_match_metric.compute(predictions=list_of_generated_recipes, references=list_of_20_examples)
results_met = meteor.compute(predictions=list_of_generated_recipes, references=list_of_20_examples)

# Calculate Bilingual Evaluation Understudy (BLEU)
results_bleu = bleu_metric.compute(predictions=list_of_generated_recipes, references=list_of_20_examples)

# **Printing evaluation metrics**

In [99]:
print("Character Error Rate for Recipe Generation:", results_CER)
print("Word Error Rate for Recipe Generation:",results_WER)
print("Exact Match for Recipe Generation:",results_em)
print("BLEU Score for Recipe Generation:",results_bleu)
print("METEOR for Recipe Generation:",results_met)

Character Error Rate for Recipe Generation: 1.5342884431709647
Word Error Rate for Recipe Generation: 1.8310961968680088
Exact Match for Recipe Generation: {'exact_match': 0.0}
BLEU Score for Recipe Generation: {'bleu': 0.10685527548350646, 'precisions': [0.26300927793465106, 0.1252541683611224, 0.07421074210742107, 0.05332782141380736], 'brevity_penalty': 1.0, 'length_ratio': 1.992765273311897, 'translation_length': 2479, 'reference_length': 1244}
METEOR for Recipe Generation: {'meteor': 0.30166785600493634}


In [100]:
from rouge_score import rouge_scorer
# Initialize the Rouge scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)

# Preprocess the text
def preprocess_text(text):
    tokens = text.split()
    return ' '.join(tokens)

# Calculate scores for each pair of predictions and references
rouge1_f1_scores = []
rouge1_precision_scores = []
rouge1_recall_scores = []
rouge2_f1_scores = []
rouge2_precision_scores = []
rouge2_recall_scores = []
rougeL_f1_scores = []
rougeL_precision_scores = []
rougeL_recall_scores = []

for ref, pred in zip(list_of_20_examples, list_of_generated_recipes):
    candidate = preprocess_text(pred)
    reference = preprocess_text(' '.join(ref))
    scores = scorer.score(reference, candidate)

    rouge1_f1_scores.append(scores['rouge1'].fmeasure)
    rouge1_precision_scores.append(scores['rouge1'].precision)
    rouge1_recall_scores.append(scores['rouge1'].recall)
    rouge2_f1_scores.append(scores['rouge2'].fmeasure)
    rouge2_precision_scores.append(scores['rouge2'].precision)
    rouge2_recall_scores.append(scores['rouge2'].recall)
    rougeL_f1_scores.append(scores['rougeL'].fmeasure)
    rougeL_precision_scores.append(scores['rougeL'].precision)
    rougeL_recall_scores.append(scores['rougeL'].recall)

# Calculate the average scores
avg_rouge1_f1 = sum(rouge1_f1_scores) / len(rouge1_f1_scores)
avg_rouge1_precision = sum(rouge1_precision_scores) / len(rouge1_precision_scores)
avg_rouge1_recall = sum(rouge1_recall_scores) / len(rouge1_recall_scores)
avg_rouge2_f1 = sum(rouge2_f1_scores) / len(rouge2_f1_scores)
avg_rouge2_precision = sum(rouge2_precision_scores) / len(rouge2_precision_scores)
avg_rouge2_recall = sum(rouge2_recall_scores) / len(rouge2_recall_scores)
avg_rougeL_f1 = sum(rougeL_f1_scores) / len(rougeL_f1_scores)
avg_rougeL_precision = sum(rougeL_precision_scores) / len(rougeL_precision_scores)
avg_rougeL_recall = sum(rougeL_recall_scores) / len(rougeL_recall_scores)

# Print the average scores
print("Average Rouge-1 F1 Score:", avg_rouge1_f1)
print("Average Rouge-1 Precision:", avg_rouge1_precision)
print("Average Rouge-1 Recall:", avg_rouge1_recall)

print("Average Rouge-2 F1 Score:", avg_rouge2_f1)
print("Average Rouge-2 Precision:", avg_rouge2_precision)
print("Average Rouge-2 Recall:", avg_rouge2_recall)

print("Average Rouge-L F1 Score:", avg_rougeL_f1)
print("Average Rouge-L Precision:", avg_rougeL_precision)
print("Average Rouge-L Recall:", avg_rougeL_recall)

Average Rouge-1 F1 Score: 0.029069723928599724
Average Rouge-1 Precision: 0.040044938060854365
Average Rouge-1 Recall: 0.02966973301740704
Average Rouge-2 F1 Score: 0.0037649484386785754
Average Rouge-2 Precision: 0.0037757545227833852
Average Rouge-2 Recall: 0.0050208141607210955
Average Rouge-L F1 Score: 0.02626064157835723
Average Rouge-L Precision: 0.036315605072322364
Average Rouge-L Recall: 0.02699192091551721
